In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import display


cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [2]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.3, 0.2, 0.2, 0.1, 0.15, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,2355.0,1570.0,1570.0,785.0,1177.50,392.50
1,2789.1,1859.4,1859.4,929.7,1394.55,464.85
2,3321.3,2214.2,2214.2,1107.1,1660.65,553.55
3,3288.0,2192.0,2192.0,1096.0,1644.00,548.00
4,3476.4,2317.6,2317.6,1158.8,1738.20,579.40
5,3557.4,2371.6,2371.6,1185.8,1778.70,592.90
6,3256.5,2171.0,2171.0,1085.5,1628.25,542.75
7,853.8,569.2,569.2,284.6,426.90,142.30
8,2770.5,1847.0,1847.0,923.5,1385.25,461.75


In [3]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Creazione di un DataFrame con tutti i tier e la std media oraria per ognuno
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]
std_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

std_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)
display(std_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_20268\345085060.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,0.000000,28.991322,42.427125,42.278934
01:00:00,0.0,0.000000,0.000000,34.521576,50.553214,50.452805
02:00:00,0.0,0.000000,0.000000,36.611001,53.669763,53.635554
03:00:00,0.0,0.000000,0.000000,38.924761,57.097983,57.000288
04:00:00,0.0,0.000032,0.046572,42.550663,62.418339,62.367183
05:00:00,0.0,0.138339,6.277810,60.382878,90.390240,90.930738
06:00:00,0.0,0.596456,31.498172,102.802546,161.313978,164.726412
07:00:00,0.0,0.960828,46.257291,133.108190,237.445624,240.651913
08:00:00,0.0,0.632438,27.723440,116.696750,213.293060,216.459444
09:00:00,0.0,0.171015,9.780120,87.361677,147.039771,162.527548


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,0.000000,0.865204,1.403410,1.424645
01:00:00,0.0,0.000000,0.000000,0.990713,1.454768,1.445946
02:00:00,0.0,0.000000,0.000000,1.057676,1.773486,1.669474
03:00:00,0.0,0.000000,0.000000,1.245038,1.784916,1.793179
04:00:00,0.0,0.000193,0.055545,1.426176,2.050683,1.988202
05:00:00,0.0,0.048147,1.236822,2.257312,3.834563,4.090113
06:00:00,0.0,0.095504,2.652671,4.242210,7.589726,7.596358
07:00:00,0.0,0.110751,3.165970,5.152397,13.336545,12.446711
08:00:00,0.0,0.096623,2.712762,4.803722,12.682301,12.466618
09:00:00,0.0,0.049724,1.583367,3.643800,6.890035,10.493887


In [4]:
result_dict = {}
for k in range(len(tier_pop_df)):  # Itero sui cluster
    pop_cluster = tier_pop_df.iloc[k, :]  # Seleziono tutto i tier per quel cluster
    nome_dataframe = f"bus_{k}"
    load_df = pd.DataFrame()
    for j in range(len(pop_cluster)):  # Itero su tutti i tier per quel cluster
        n_person = int(pop_cluster[j])
        mean_load_person = mean_demand_tier_df.iloc[:, j].values
        total_load = pd.Series(n_person * mean_load_person)
        load_df[f"tier_{j}"] = total_load

    result_dict[nome_dataframe] = load_df

result_dict["bus_2"]

,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
0,0.0,0.000000,0.000000,32093.393788,70429.027767,23380.250399
1,0.0,0.000000,0.000000,38215.384288,83918.335864,27900.401219
2,0.0,0.000000,0.000000,40528.378154,89091.806319,29660.461569
3,0.0,0.000000,0.000000,43089.709895,94782.651979,31521.159376
4,0.0,0.071619,103.111119,47103.584396,103614.443446,34489.052199
5,0.0,306.283075,13899.070717,66843.845514,150047.797821,50284.698378
6,0.0,1320.554576,69736.953746,113802.418826,267781.203311,91093.705586
7,0.0,2127.272576,102413.642544,147350.766393,394159.735077,133080.507635
8,0.0,1400.217612,61379.695721,129183.301935,354066.479424,119702.072570
9,0.0,378.626632,21653.185275,96709.376388,244086.019323,89877.734175


In [5]:
tot_result_dict = {}
for key in result_dict:
    nome_dataframe = f"{key}"
    load = result_dict[key]
    load_tot = pd.DataFrame(load.sum(axis=1))
    load_tot.rename(columns={0: key}, inplace=True)
    tot_result_dict[nome_dataframe] = load_tot

tot_result_dict["bus_1"]

,bus_1
0,105693.776182
1,125951.826220
2,133714.166601
3,142203.824717
4,155565.742460
5,236219.170938
6,456472.220794
7,654097.678128
8,558892.564758
9,380044.379733


In [6]:
tot_loads_df = pd.DataFrame()
for key, cluster_load in tot_result_dict.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [tot_loads_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_20268\2838391707.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
2013-01-01 00:00:00,89268.256376,105693.776182,1.259027e+05,1.246935e+05,1.317898e+05,1.348193e+05,1.234421e+05,32311.099454,105011.147321,91927.353376
2013-01-01 01:00:00,106378.069835,125951.826220,1.500341e+05,1.485933e+05,1.570496e+05,1.606597e+05,1.471020e+05,38504.095144,125138.359422,109546.862240
2013-01-01 02:00:00,112934.083999,133714.166601,1.592806e+05,1.577510e+05,1.667286e+05,1.705611e+05,1.561678e+05,40877.091988,132850.566066,116298.205375
2013-01-01 03:00:00,120104.376115,142203.824717,1.693935e+05,1.677668e+05,1.773143e+05,1.813902e+05,1.660830e+05,43472.413721,141285.393441,123682.078474
2013-01-01 04:00:00,131389.761348,155565.742460,1.853103e+05,1.835307e+05,1.939753e+05,1.984342e+05,1.816887e+05,47557.259047,154561.012620,135303.618498
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,813935.450607,963758.541701,1.147972e+06,1.136970e+06,1.201728e+06,1.229328e+06,1.125590e+06,294661.500900,957534.254698,838203.983188
2013-12-31 20:00:00,649553.446510,769090.315028,9.161083e+05,9.072799e+05,9.589363e+05,9.810097e+05,8.982287e+05,235168.038602,764123.454308,668870.576874
2013-12-31 21:00:00,494138.998865,585068.852430,6.969121e+05,6.901842e+05,7.294762e+05,7.462798e+05,6.833067e+05,178905.431767,581290.471952,508821.904437
2013-12-31 22:00:00,304829.412365,360924.339440,4.299178e+05,4.257668e+05,4.500069e+05,4.603735e+05,4.215262e+05,110361.177645,358593.499317,313886.469948


# CON DEVIAZIONE STANDARD:

In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import display


cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [8]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.3, 0.2, 0.2, 0.1, 0.15, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,2355.0,1570.0,1570.0,785.0,1177.50,392.50
1,2789.1,1859.4,1859.4,929.7,1394.55,464.85
2,3321.3,2214.2,2214.2,1107.1,1660.65,553.55
3,3288.0,2192.0,2192.0,1096.0,1644.00,548.00
4,3476.4,2317.6,2317.6,1158.8,1738.20,579.40
5,3557.4,2371.6,2371.6,1185.8,1778.70,592.90
6,3256.5,2171.0,2171.0,1085.5,1628.25,542.75
7,853.8,569.2,569.2,284.6,426.90,142.30
8,2770.5,1847.0,1847.0,923.5,1385.25,461.75


In [9]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Creazione di un DataFrame con tutti i tier e la std media oraria per ognuno
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"tier_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]
std_demand_tier_df.insert(0, "tier_0", np.zeros(len(mean_demand_tier_df)))

std_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)
display(std_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_20268\345085060.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,0.000000,28.991322,42.427125,42.278934
01:00:00,0.0,0.000000,0.000000,34.521576,50.553214,50.452805
02:00:00,0.0,0.000000,0.000000,36.611001,53.669763,53.635554
03:00:00,0.0,0.000000,0.000000,38.924761,57.097983,57.000288
04:00:00,0.0,0.000032,0.046572,42.550663,62.418339,62.367183
05:00:00,0.0,0.138339,6.277810,60.382878,90.390240,90.930738
06:00:00,0.0,0.596456,31.498172,102.802546,161.313978,164.726412
07:00:00,0.0,0.960828,46.257291,133.108190,237.445624,240.651913
08:00:00,0.0,0.632438,27.723440,116.696750,213.293060,216.459444
09:00:00,0.0,0.171015,9.780120,87.361677,147.039771,162.527548


,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
00:00:00,0.0,0.000000,0.000000,0.865204,1.403410,1.424645
01:00:00,0.0,0.000000,0.000000,0.990713,1.454768,1.445946
02:00:00,0.0,0.000000,0.000000,1.057676,1.773486,1.669474
03:00:00,0.0,0.000000,0.000000,1.245038,1.784916,1.793179
04:00:00,0.0,0.000193,0.055545,1.426176,2.050683,1.988202
05:00:00,0.0,0.048147,1.236822,2.257312,3.834563,4.090113
06:00:00,0.0,0.095504,2.652671,4.242210,7.589726,7.596358
07:00:00,0.0,0.110751,3.165970,5.152397,13.336545,12.446711
08:00:00,0.0,0.096623,2.712762,4.803722,12.682301,12.466618
09:00:00,0.0,0.049724,1.583367,3.643800,6.890035,10.493887


In [10]:
result_dict = {}
for k in range(len(tier_pop_df)):  # Itero sui cluster
    pop_cluster = tier_pop_df.iloc[k, :]  # Seleziono tutto i tier per quel cluster
    nome_dataframe = f"bus_{k}"
    load_df = pd.DataFrame()
    std_df = pd.DataFrame()
    for j in range(len(pop_cluster)):  # Itero su tutti i tier per quel cluster
        n_person = int(pop_cluster[j])
        mean_load_person = mean_demand_tier_df.iloc[:, j].values
        mean_load = pd.Series(n_person * mean_load_person)

        sqrt_n_person = np.sqrt(n_person)
        std_load_person = std_demand_tier_df.iloc[:, j].values
        std_load = np.random.normal(0, std_load_person) * sqrt_n_person
        std_total = pd.Series(std_load)

        total_load = pd.Series(mean_load.values + std_total.values)
        load_df[f"tier_{j}"] = total_load

    result_dict[nome_dataframe] = load_df

display(result_dict["bus_3"])

,tier_0,tier_1,tier_2,tier_3,tier_4,tier_5
0,0.0,0.000000,0.000000,31775.695189,69709.786308,23171.210677
1,0.0,0.000000,0.000000,37850.708216,83198.014619,27695.368018
2,0.0,0.000000,0.000000,40145.253298,88291.701913,29425.481206
3,0.0,0.000000,0.000000,42622.460186,93836.897880,31291.814506
4,0.0,0.062067,102.285046,46696.475977,102677.221029,34233.249621
5,0.0,299.693264,13779.760947,66326.948462,148649.127496,49858.121900
6,0.0,1300.628290,69097.317966,112730.484136,265385.786044,90294.276908
7,0.0,2109.581420,101396.062637,145515.771771,390937.355295,131982.068864
8,0.0,1384.172382,60514.061244,127987.467793,350922.080950,119024.922670
9,0.0,377.958870,21371.249461,95815.820506,242089.906421,89143.065995


In [11]:
tot_result_dict = {}
for key in result_dict:
    nome_dataframe = f"{key}"
    load = result_dict[key]
    load_tot = pd.DataFrame(load.sum(axis=1))
    load_tot.rename(columns={0: key}, inplace=True)
    tot_result_dict[nome_dataframe] = load_tot

tot_result_dict["bus_1"]

,bus_1
0,105732.233888
1,126066.574460
2,133828.408141
3,142139.030044
4,155620.515042
5,236172.672903
6,456614.238941
7,653436.065453
8,558178.408804
9,380285.923010


In [12]:
tot_loads_df = pd.DataFrame()
for key, cluster_load in tot_result_dict.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [tot_loads_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_20268\2838391707.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
2013-01-01 00:00:00,89215.503287,105732.233888,1.258727e+05,1.246567e+05,1.317899e+05,1.348324e+05,1.233659e+05,32346.623686,105016.583197,92108.423397
2013-01-01 01:00:00,106404.195869,126066.574460,1.500908e+05,1.487441e+05,1.569489e+05,1.608672e+05,1.471002e+05,38540.037840,125234.216359,109591.131576
2013-01-01 02:00:00,112952.370055,133828.408141,1.592156e+05,1.578624e+05,1.667928e+05,1.704536e+05,1.560836e+05,40855.894410,132907.261048,116244.426311
2013-01-01 03:00:00,119974.418852,142139.030044,1.693370e+05,1.677512e+05,1.773423e+05,1.814327e+05,1.661437e+05,43411.427316,141227.246293,123655.990624
2013-01-01 04:00:00,131365.627003,155620.515042,1.852313e+05,1.837093e+05,1.938858e+05,1.983429e+05,1.815926e+05,47644.706084,154634.731540,135331.833405
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,813904.499394,962219.647893,1.148367e+06,1.135099e+06,1.202753e+06,1.227203e+06,1.124643e+06,294255.667037,957086.511623,839316.413653
2013-12-31 20:00:00,650044.843203,768736.886533,9.164294e+05,9.082724e+05,9.595119e+05,9.804447e+05,8.983826e+05,234251.775078,763866.938315,668644.686510
2013-12-31 21:00:00,494340.496870,584673.394141,6.974009e+05,6.896838e+05,7.284110e+05,7.466225e+05,6.838095e+05,178711.398223,582009.399127,508969.091166
2013-12-31 22:00:00,304752.273942,361045.956374,4.302618e+05,4.256837e+05,4.502451e+05,4.608192e+05,4.216323e+05,110506.436650,358428.885981,313930.527158
